### Set up

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [3]:
llm_generate_writing_assignment = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.8)
llm_story_emulator = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.7)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.8)
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.8)

### Generate writing assignment

In [4]:
writing_assignment = llm_generate_writing_assignment('Give a prompt for a creative writing assignment')
print(writing_assignment)



Write a story about a group of friends who stumble upon a mysterious portal that leads to a parallel universe. 


### Generate a draft

In [5]:
draft = llm_story_emulator(writing_assignment + "with only 3 paragraphs")
print(draft)



It was a typical summer day for a group of four friends – Alex, Sarah, Jack, and Lily. They had decided to go on a hike in the nearby woods, hoping to find a new adventure. As they wandered deeper into the forest, they stumbled upon a strange-looking rock formation. Curiosity getting the best of them, they approached the rocks and noticed a shimmering light coming from behind them.

Without hesitation, they climbed over the rocks and were amazed to find a mysterious portal on the other side. The portal seemed to be emitting a soft humming sound and pulsating with a blue light. Excited and intrigued, the friends decided to step through the portal and see where it led them. As soon as they stepped through, they found themselves in a lush and vibrant forest unlike anything they had ever seen before. They couldn't believe their eyes – they had stumbled upon a parallel universe. Little did they know, this was just the beginning of their extraordinary adventure.


### Construct the prompt and get feedback

#### Use GPT-3.5-turbo-instruct model

In [6]:
prompt = f"""Imagine you are a writing coach collaborating with a student on a creative project: {writing_assignment}.
The student has started a story, essay, or any piece of writing, and they need your guidance to enhance their work.
Provide constructive feedback, suggestions, and ideas to help the student improve their writing.
Focus on areas such as clarity, organization, creativity, and language use.
Point out strengths in their writing and offer specific advice on how they can build on those strengths.
Encourage them to explore new perspectives, expand on their ideas, and refine their arguments.
Your goal is to empower the student to develop their writing skills by providing thoughtful and insightful guidance.
Be supportive, encouraging, and specific in your feedback, helping the student to become a better writer.
Here is the student's draft: {draft}
"""

print(llm(prompt))


Great start to your story! You have set up the scene and introduced the main characters in a clear and concise manner. However, there are a few areas where you can improve and build upon your writing.

Firstly, consider adding more sensory details to your description of the mysterious portal. How did it feel to touch the rocks? What did the shimmering light look like? How did the humming sound make the friends feel? These details will help the reader to fully immerse themselves in the world you are creating and enhance the sense of curiosity and wonder that the characters are experiencing.

You can also expand on the characters' reactions to the portal and their decision to step through it. How did they feel? Were they scared, excited, or hesitant? This will help the reader to better understand the characters' motivations and emotions in this pivotal moment.

Additionally, it would be beneficial to provide more context about the friends' relationship and their personalities. This will

#### Use GPT-3.5-turbo (chat) model

In [11]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
messages = [
    SystemMessage(
        content="""Imagine you are a writing coach collaborating with a student on a creative project.
        The student has started a story, essay, or any piece of writing, and they need your guidance to enhance their work.
        Provide constructive feedback, suggestions, and ideas to help the student improve their writing.
        Focus on areas such as clarity, organization, creativity, and language use.
        Point out strengths in their writing and offer specific advice on how they can build on those strengths.
        Encourage them to explore new perspectives, expand on their ideas, and refine their arguments.
        Your goal is to empower the student to develop their writing skills by providing thoughtful and insightful guidance.
        Be supportive, encouraging, and specific in your feedback, helping the student to become a better writer.
        """
    ),
    HumanMessage(
        content=f"""Hi, here is the prompt that I'm writing about: {writing_assignment}
        Here is what I've written so far. {draft}
        Please assist me in my writing process without writing on behalf of me.
        """
    ),
]
print(chat(messages).content)

Great start to your story! You've set up the scene and introduced the main conflict - the discovery of a mysterious portal. Here are some suggestions to enhance your writing and further develop your story:

1. Begin with a captivating hook: While your opening sentence is good, you can make it even more attention-grabbing. Consider starting with an action or an intriguing detail to immediately engage the reader. For example, "As the summer sun beat down on their backs, the group of friends ventured deep into the unknown woods, craving an adventure that would forever change their lives."

2. Build anticipation and curiosity: After the friends stumble upon the rocks, heighten the suspense by describing their thoughts and emotions in more detail. How do they feel as they notice the shimmering light? What thoughts are racing through their minds? This will make the reader feel more connected to the characters and eager to know what happens next.

3. Show, don't tell: When you mention that th

### Prepare the rubric

In [9]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("")

documents = loader.load()

In [10]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')

Average length among 1 documents loaded is 4970 characters.


Mannally extract key component (descriptors) from Narrative_rubric.pdf and saved them in a text file.

In [11]:
with open("rubric.txt") as file:
    rubric = file.read()

In [12]:
print(rubric)

Descriptors:

-Ideas
Strong and creative story idea; specific, descriptive details; use of plot, setting, characters, conflict, and dialogue 

-Organization
Unified structure, clear direction, and clever transitions 

-Word Choice
Precise, rich language that expresses ideas and engages the reader

-Sentence Fluency and Voice
Rhythmic and flowing language, varied sentences, and unique perspective with ideas and details to appeal to the audience

-Conventions
Mechanical and grammatical accuracy


In [13]:
prompt = f"""You are a helpful assistant for a student writing a story based on the following writing assignment: {writing_assignment}.
             Use concise expression to provide constructive pointers and suggestions to help the student improve their writing.
             Focus on the descriptors in the following rubric and reference the rubric: {rubric}. 
             The following is the student's writing: {story}
             Avoid writing for the student.
             """

print(llm(prompt))


Overall, your story idea is strong and creative. The concept of humans being able to communicate with animals is very intriguing. However, in order to make the story more engaging, try to provide more specific and descriptive details. This will help readers to visualize the world you are creating.

To improve the organization of your story, consider adding a clear conflict or obstacle that the characters must face. This will create a more unified structure and provide a clear direction for the plot. Also, try to use clever transitions between scenes or events to keep the story flowing smoothly.

In terms of word choice, use precise and rich language that will express your ideas effectively and engage the reader. You can also try to incorporate dialogue between characters to add depth to the story.

For sentence fluency and voice, focus on creating a rhythmic and flowing language. Vary your sentence structure and use a unique perspective to appeal to your audience.

Lastly, pay attenti